In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData, text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
load_dotenv()
PARQUET_PATH = os.getenv("PARQUET_PATH")
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)

In [3]:
query = text('''
            select c.ContactPersoonId, i.CampagneId, i.CampagneNaam, ca.Startdatum, a.Ondernemingsaard, a.Ondernemingstype, a.PrimaireActiviteit, f.Naam as Functie
            from Contactfiche c
            join Account a on a.AccountId = c.AccountId
            join Inschrijving i on i.ContactficheId = c.ContactPersoonId
            join Campagne ca on ca.CampagneId = i.CampagneId
            join ContactficheFunctie cf on cf.ContactpersoonId = c.ContactPersoonId
            join Functie f on f.FunctieId = cf.FunctieId
            where i.CampagneId is not null and a.status = 'actief';
            ''')
result = conn.execute(query)

#convert to dataframe
df_omschrijving = pd.DataFrame(result.fetchall())
df_omschrijving["rating"] = 5

ddf_omschrijving = dd.from_pandas(df_omschrijving, npartitions=10)

del df_omschrijving


query2 = text('''
                with pageview_count as(
                    select p.PageTitle, count(p.ContactId) as count
                    from Pageviews p
                    group by p.PageTitle
                )
                select PageTitle, ContactId
                from Pageviews
                where Pagetitle in (select top (2500) PageTitle
                from pageview_count
                order by count desc)
              ''')

result2 = conn.execute(query2)
df_pageviews = pd.DataFrame(result2.fetchall())
df_pageviews["rating"] = 1

ddf_pageviews = dd.from_pandas(df_pageviews, npartitions=20)

del df_pageviews

In [4]:
ddf_campagne = ddf_omschrijving.iloc[:, 0:4]
display(ddf_campagne.head())
ddf_campagne = ddf_campagne[ddf_campagne['Startdatum'] > '2022-11-06']
ddf_campagne['rating'] = 10
ddf_omschrijving = ddf_omschrijving.drop(ddf_omschrijving.columns[[1,2,3]], axis=1)
ddf_omschrijving.drop_duplicates(inplace=True)
display(ddf_omschrijving.head())

,ContactPersoonId,CampagneId,CampagneNaam,Startdatum
0,9E22A289-C6F3-ED11-8849-6045BD895CDC,044F17B7-A8B6-ED11-83FF-6045BD895CDC,OV-JO-Stamgasten 2023,2023-05-23 19:00:00
1,12C6CFE5-5020-EC11-8124-001DD8B72B61,0A14941A-711C-EC11-8123-001DD8B72B61,OV-Community Bouw en Vastgoed 2022,2022-01-31 08:00:00
2,EC6D5B41-BA4A-E711-80E7-001DD8B72B61,1C66F294-192D-E811-80F0-001DD8B72B61,OV-NW-Voka Update-Big Refresh-Hervorming innov...,2018-02-26 06:00:00
3,6CE844CB-18AD-ED11-AAD0-6045BD895BFB,31E7F5EB-15FA-ED11-8849-6045BD8952D3,OV-Effectief leiderschap-nj 2023,2023-11-21 09:00:00
4,01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018,2018-01-18 06:00:00


,ContactPersoonId,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
0,9E22A289-C6F3-ED11-8849-6045BD895CDC,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Financieel,5
1,12C6CFE5-5020-EC11-8124-001DD8B72B61,Diensten,Familiebedrijf,Bouw,Verantwoordelijke Commercieel,5
2,EC6D5B41-BA4A-E711-80E7-001DD8B72B61,Diensten,Bedrijf,Grafische industrie en diensten,Bedrijfsleider,5
3,6CE844CB-18AD-ED11-AAD0-6045BD895BFB,Diensten,Bedrijf,Overige industrie & diensten,Verantwoordelijke Commercieel,5
4,01218324-0942-E611-80D6-005056B06EC4,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,5


In [5]:
ddf_pageviews.to_parquet(PARQUET_PATH + "dfpageviews.parquet")
ddf_omschrijving.to_parquet(PARQUET_PATH + "dfomschrijving.parquet")
ddf_campagne.to_parquet(PARQUET_PATH + "dfcampagne.parquet")


In [6]:
df_pivot_omschrijving = pd.pivot_table(df_omschrijving, index='ContactPersoonId', columns=['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit', 'Functie'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_omschrijving.info()

NameError: name 'df_omschrijving' is not defined

In [ ]:
df_pivot_campagne = pd.pivot_table(df_campagne, index='ContactPersoonId', columns=['CampagneId'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_campagne.info()

NameError: name 'df_campagne' is not defined

In [ ]:
df_pivot_pageviews = pd.pivot_table(df_pageviews, index='ContactId', columns=['PageTitle'], values='rating', fill_value = 0).fillna(0).astype('int8')
df_pivot_pageviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18505 entries, 00013C8A-6F1C-E211-9DAA-005056B06EB4 to FFD685FA-1F7B-ED11-81AD-6045BD895CDC
Columns: 2499 entries, "Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!" | Voka to | Voka
dtypes: int8(2499)
memory usage: 44.2+ MB


In [ ]:
df_pivot = pd.concat([df_pivot_omschrijving, df_pivot_pageviews, df_pivot_campagne], axis=1, join='outer').fillna(0).astype('int8')
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25111 entries, 00223C8E-467F-E311-BBFD-005056B06EB4 to FFFEA9CB-ED93-EC11-B400-000D3A2B10EB
Columns: 6341 entries, ('Diensten', 'Bedrijf', 'Accountancy & boekhouding', 'Bedrijfsleider') to FFE56B2D-B848-EE11-BE6E-6045BD895D78
dtypes: int8(6341)
memory usage: 152.0+ MB
